In [35]:
#Import das bibliotecas usadasergtfdfg
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

In [ ]:
#Carregamnto dos datasets
dataframe = pd.read_csv("../datasets/dataset.csv")
dataframe_features = pd.read_csv("../datasets/just_features.csv")

print(dataframe)


#Organização dos dataframes
#Para o dataframe
dataframe.set_index(dataframe['id'], inplace=True)
dataframe.pop('id')
dataframe_target = dataframe.pop('preco')

#Para o dataframe_features
dataframe_features.set_index(dataframe_features['id'], inplace=True)
dataframe_features.pop('id')

##print(dataframe)
##print(dataframe_features)

#Padrão One-Hot Encoding para as variáveis categóricas
for x in ['tipo','combustivel','marca']:
    dataframe_x = pd.get_dummies(dataframe[x])
    dataframe = pd.concat([dataframe, dataframe_x], axis=1)
    dataframe.pop(x)

    dataframe_features_x = pd.get_dummies(dataframe_features[x])
    dataframe_features = pd.concat([dataframe_features, dataframe_features_x], axis=1)
    dataframe_features.pop(x)

#Por simplicidade a coluna 'caixa' será substituída por 'automatico' caso o valor seja True é automático caso False é manual
dataframe.rename(columns={'caixa':'automatico'}, inplace=True)
dataframe_features.rename(columns={'caixa':'automatico'}, inplace=True)
mapeamento = {'automatica': True, 'manual': False}
dataframe['automatico'] = dataframe['automatico'].map(mapeamento)
dataframe_features['automatico'] = dataframe_features['automatico'].map(mapeamento)

#A coluna com_danos será remapeada para True e False támbem
mapeamento = {'sim': True, 'nao': False}
dataframe['com_danos'] = dataframe['com_danos'].map(mapeamento)
dataframe_features['com_danos'] = dataframe_features['com_danos'].map(mapeamento)

##print(dataframe)
##print(dataframe_features)

id             0
tipo           0
ano_registo    0
caixa          0
potencia       0
km             0
combustivel    0
marca          0
com_danos      0
preco          0
dtype: int64
id             0
tipo           0
ano_registo    0
caixa          0
potencia       0
km             0
combustivel    0
marca          0
com_danos      0
dtype: int64
            tipo  ano_registo       caixa  potencia      km combustivel  \
id                                                                        
26713  limousine         2014      manual       140   40800    gasolina   
49553  limousine         2013      manual       184   30300      diesel   
9007       coupe         2009      manual       177  150900      diesel   
21152   compacto         2006      manual        54  150500    gasolina   
37819  limousine         2015      manual       150   40600      diesel   
...          ...          ...         ...       ...     ...         ...   
28021  limousine         2014  automatica       140

In [43]:
#Pré processamento
#Verificação de valores Nan no dataset
dataframe_null = dataframe.isnull().sum()
dataframe_features_null = dataframe_features.isnull().sum()

#print(dataframe_null)
#print(dataframe_features_null)

#Veriificação de valores repetidos no dataset
duplicados = dataframe.duplicated(keep=False)
print(duplicados.to_string())
#Identificação e remoção de outliers
#Serão levadas em consideração as colunas potencia e km
#Potencia


# q1,q3 = np.percentile(dataframe.potencia,[25,75])
# lim_if = q1 -1.5*(q3-q1)
# lim_sup = q3 +1.5*(q3-q1)
# print(q1,q3,lim_if,lim_sup)
# print(dataframe[(dataframe.potencia<lim_if)|(dataframe.potencia>lim_sup)])

# q1,q3 = np.percentile(dataframe.km,[25,75])
# lim_if = q1 -1.5*(q3-q1)
# lim_sup = q3 +1.5*(q3-q1)
# print(q1,q3,lim_if,lim_sup)
# print(dataframe[(dataframe.km<lim_if)|(dataframe.km>lim_sup)])


id
26713    False
49553    False
9007     False
21152    False
37819    False
47700    False
1246     False
46110    False
47496    False
78       False
3555      True
81679    False
12414    False
60448    False
70948    False
23112    False
1875     False
76611    False
33371    False
94383    False
3384     False
74015    False
84567    False
74251    False
24741    False
72166    False
79062    False
86831    False
53818     True
62152     True
5586     False
13254    False
16104    False
71051    False
68171    False
75730    False
21741    False
83684    False
4198     False
20816    False
47469    False
92838    False
49168    False
2836     False
24283    False
51487    False
921      False
79385    False
9563     False
52412    False
79704    False
33376    False
73253    False
81783    False
87231    False
37330    False
54708    False
85912    False
14655    False
73760    False
87065    False
26202    False
31130    False
93794    False
94124    False
13373    False
33755  

In [38]:
#Pré processamento
#Normalização

colunasnum = dataframe.select_dtypes(include='number')
scaler = MinMaxScaler()
valores_normalizados = scaler.fit_transform(colunasnum)
dataframe[colunasnum.columns] = valores_normalizados

colunasnum = dataframe_features.select_dtypes(include='number')
scaler = MinMaxScaler()
valores_normalizados = scaler.fit_transform(colunasnum)
dataframe_features[colunasnum.columns] = valores_normalizados

##print(dataframe)
##print(dataframe_features)
dataframe

,ano_registo,automatico,potencia,km,com_danos,cabrio,carrinha,compacto,coupe,kombi,...,audi,bmw,fiat,ford,mercedes_benz,opel,outra,peugeot,renault,volkswagen
id,,,,,,,,,,,,,,,,,,,,,
26713,0.823529,False,0.008400,0.245205,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
49553,0.764706,False,0.011098,0.173288,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
9007,0.529412,False,0.010669,0.999315,False,False,False,False,True,False,...,False,True,False,False,False,False,False,False,False,False
21152,0.352941,False,0.003127,0.996575,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,True
37819,0.882353,False,0.009013,0.243836,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28021,0.823529,True,0.008400,0.106849,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
1903,0.176471,False,0.008829,0.994521,False,True,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
77063,0.411765,False,0.004721,0.447945,False,False,False,True,False,False,...,False,False,False,False,False,False,True,False,False,False
